In [1]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 80.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22

In [2]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -qqq -e . --progress-bar off

Cloning into 'mergekit'...
remote: Enumerating objects: 1114, done.
remote: Counting objects: 100% (394/394), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1114 (delta 335), reused 269 (delta 263), pack-reused 720
Receiving objects: 100% (1114/1114), 272.89 KiB | 22.74 MiB/s, done.
Resolving deltas: 100% (783/783), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done


In [3]:
MODEL_NAME = "Frankenmerge_WizardLM"
yaml_config = """

dtype: float16
merge_method: passthrough
slices:
- sources:
  - layer_range: [0, 20]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [10, 30]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [20, 40]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [30, 50]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [40, 60]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [50, 70]
    model: WizardLM/WizardLM-70B-V1.0
- sources:
  - layer_range: [60, 80]
    model: WizardLM/WizardLM-70B-V1.0
"""

In [4]:
# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [ ]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "luqmanxyz"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "luqmanxyz"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("hf"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)